In [1]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), ".")))

In [ ]:
from datasets import load_dataset
from sklearn.metrics import classification_report
import numpy as np
from trc_model.temporal_relation_classification import TemporalRelationClassification
from trc_model.temporal_relation_classification_config import TemporalRelationClassificationConfig
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding, AutoTokenizer
from collections import Counter

In [ ]:
def calculate_class_weights(dataset):
    labels = dataset['train']['label']
    labels_count = Counter(labels)
    class_weights = [0] * len(labels_count)
    for l, count in labels_count.items():
        cls_w = 1 - (count / len(labels))
        class_weights[l] = cls_w
    return class_weights

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
eval_mode = False


def compute_metrics(eval_preds):
    predictions, labels = eval_preds
    predictions = np.argmax(predictions, axis=1)

    if eval_mode:
        report = classification_report(y_true=labels, y_pred=predictions,
                                       target_names=['BEFORE', 'AFTER', 'EQUAL', 'VAGUE'])
        for i in range(labels.shape[0]):
            if labels[i] == 3 and predictions[i] != 3:
                labels[i] = predictions[i]
        report_no_vague = classification_report(y_true=labels, y_pred=predictions,
                                                target_names=['BEFORE', 'AFTER', 'EQUAL', 'VAGUE'])

        with open(f'{model_final_name}/evaluation_report.txt', 'w') as f:
            f.write(report)
            f.write('\n')
            f.write(report_no_vague)
        print(report)
        print(report_no_vague)

    results = \
        classification_report(y_true=labels, y_pred=predictions, target_names=['BEFORE', 'AFTER', 'EQUAL', 'VAGUE'],
                              output_dict=True)
    final_results = results['weighted avg']
    final_results.pop('support')
    final_results['BEFORE-f1'] = results['BEFORE']['f1-score']
    final_results['AFTER-f1'] = results['AFTER']['f1-score']
    final_results['EQUAL-f1'] = results['EQUAL']['f1-score']
    final_results['VAGUE-f1'] = results['VAGUE']['f1-score']
    return final_results

In [ ]:
lm_checkpoints = ['imvladikon/alephbertgimmel-base-512', 'bert-base-multilingual-cased']
architectures = ['ESS', 'EMP', 'SEQ_CLS']

In [ ]:
raw_datasets = load_dataset(f"data_handling/data_splits/split_3")
label2id = {}
id2label = {}
for label, named_label in zip(raw_datasets['train']['label'], raw_datasets['train']['named_label']):
    label2id[named_label] = label
    id2label[label] = named_label

class_weights = calculate_class_weights(raw_datasets)
class_weights

In [ ]:
for checkpoint in lm_checkpoints:
    for arc in architectures:
        model_base_name = checkpoint.split("/")[1] if "/" in checkpoint else checkpoint
        model_final_name = f'hebrew-trc-{model_base_name}-{arc}'
        tokenizer = AutoTokenizer.from_pretrained(checkpoint)
        tokenizer.add_special_tokens({'additional_special_tokens': ['[א1]', '[/א1]', '[א2]', '[/א2]']})
        E1_start = tokenizer.convert_tokens_to_ids('[א1]')
        E1_end = tokenizer.convert_tokens_to_ids('[/א1]')
        E2_start = tokenizer.convert_tokens_to_ids('[א2]')
        E2_end = tokenizer.convert_tokens_to_ids('[/א2]')
        tokenized_datasets = raw_datasets.map(preprocess_function, remove_columns=['named_label'], batched=True)
        data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

        tokenizer_class = str(type(tokenizer)).strip("><'").split('.')[-1]
        config = TemporalRelationClassificationConfig(EMS1=E1_start,
                                                      EMS2=E2_start,
                                                      EME1=E1_end,
                                                      EME2=E2_end,
                                                      class_weights=class_weights,
                                                      architecture=arc,
                                                      num_labels=len(label2id),
                                                      id2label=id2label,
                                                      label2id=label2id,
                                                      name_or_path=checkpoint,
                                                      tokenizer_class=tokenizer_class,
                                                      vocab_size=len(tokenizer))

        model = TemporalRelationClassification(config=config)

        training_args = TrainingArguments(
            output_dir=model_final_name,
            learning_rate=3e-5,
            per_device_train_batch_size=32,
            per_device_eval_batch_size=32,
            weight_decay=0.01,
            num_train_epochs=10,
            evaluation_strategy="epoch",
            save_strategy="no",
            optim='adamw_torch',
            report_to=[],
            use_mps_device=True
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=tokenized_datasets["train"],
            eval_dataset=tokenized_datasets["test"],
            tokenizer=tokenizer,
            data_collator=data_collator,
            compute_metrics=compute_metrics,
        )
        trainer.train()
        eval_mode = True
        print('Evaluate:', model_final_name)
        trainer.evaluate(tokenized_datasets['test'])
        eval_mode = False
        config.register_for_auto_class()
        model.register_for_auto_class('AutoModelForSequenceClassification')
        # trainer.push_to_hub()
        trainer.save_model(model_final_name)